In [1]:
excercise_dict={
   "squat":
    {
        
        "knee":
        {
            "extension":"",
            "flexion":"bend",
             "hFlexion":"",
            "abduction":"",
            "angle1":20,
            "angle2":160
            
        }     
        
    },
    "sumosquat":
    {
        
        "knee":
        {
            "extension":"",
            "flexion":"bend",
            "hFlexion":"",
            "abduction":"",
            "angle1":75,
            "angle2":180
            
        }     
        
    },
    
    "sidekick":
    {
        
        "rhip":
        {
            "extension":"",
            "flexion":"",
             "hFlexion":"",
            "abduction":"move",
            "angle1":90,
            "angle2":180
            
        }     
        
    },
     "shoulderpress":
    {
        
        "shoulder":
        {
            "extension":"straighten",
            "flexion":"",
             "hFlexion":"",
            "abduction":"",
            "angle1":90,
            "angle2":180
            
        }     
        
    },
     "benchpress":
    {
        
        "shoulder":
        {
            "extension":"",
            "flexion":"",
             "hflexion":"push",
            "abduction":"",
            "angle1":45,
            "angle2":90
            
        }     
        
    }  
}

In [2]:
for x in excercise_dict["benchpress"]["shoulder"].values():
    print(x)



push

45
90


In [2]:
import pandas as pd

In [29]:
def keygenerator(data):
    angle=data[0]
    joint=data[1]
    excercise=data[2]
    if angle<excercise_dict[excercise][joint]["angle1"] or angle>excercise_dict[excercise][joint]["angle1"]:
        if angle<excercise_dict[excercise][joint]["angle1"]: 
            keywords="straighten "+joint
        else:
            keywords="bend "+joint
    else:
        keywords="good work"
        
    return keywords
    

In [ ]:
def keywordgen(data):
    angle=data[0]
    joint=data[1]
    excercise=data[2]
    if angle>excercise_dict[excercise][joint]["angle1"]:
        keyword= joint+" "+str(excercise_dict[excercise][joint][x] for x in 
                               
    
    

# Loading

In [3]:
import autocompleter 
autocompl = autocompleter.Autocompleter()

In [3]:
df = autocompl.import_json("sample-test.json")
df.shape, df.columns

load json file...
(19, 3)


C:\Users\chari\Text-completion-NLP\autocompleter.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  column_as_df = json_normalize(df[column])


((19, 3), Index(['IsFromCustomer', 'Text', 'index'], dtype='object'))

The file contains 22K conversations between a customer and a representative.
For the purpose of this project, we are only interested in completing the threads of the representative.

In [4]:

df.head(10)

,IsFromCustomer,Text,index
0,False,well done continue your excercise,0
1,False,perfectly done continue excercise,0
2,False,good work continue excercise,0
3,False,for bicepcurl angle less for backbone PLease k...,0
4,False,for lunges angle less for backbone PLease kee...,0
5,False,angle less for legs,0
6,False,angle less for hand,0
7,False,angle less for hand PLease keep your arms stra...,0
8,False,angle good for arms,0
9,False,for bicepcurl angle less for neck PLease keep ...,0


In [4]:
df1=pd.read_csv('presentences3.csv')
df1.tail()

,IsFromCustomer,Text,index
998,False,straighten arms upwards,0
999,False,align upwards,0
1000,False,straighten hips sideways,0
1001,False,straighten knees sideways,0
1002,False,good well done,0


# Data Selection and Cleaning

The data is going to separate the threads from the customer and the representative, separate the sentenses based on the punctuation (we will keep the punctuation), the final text will be cleaned up with some light regex and only the sentense larger than 1 word will be kept.

Finally, since the representative has the tendency to ask the same question over and over again, the autocomplete is extremely useful by suggesting a complete sentense. In our case, we will count the number of occurence of the same sentense so we can use it as a feature later on and delete the duplicates.

In [5]:
new_df = autocompl.process_data(df1)
new_df.shape, new_df.columns

select representative threads...
split sentenses on punctuation...
Text Cleaning using simple regex...
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(174, 5)


((174, 5),
 Index(['IsFromCustomer', 'Text', 'index', 'nb_words', 'Counts'], dtype='object'))

# Model and TFIDF matrix

A matrice of similarity is calculated based on the frequency of all the words in the data using tfidfvectorizer

In [6]:
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)

tfidf_matrice  (174, 291)


# Ranking Function

Finally, the autocomplete is calculating the similarity between the sentense in the data and the prefix of the sentense written by the representative. As a weight feature, we chose to reorder using the frequency of the most common similar sentense.

examples of auto completions

In [9]:
prefix = newgen([160,"hips","sidekick"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)[0]

hips straighten sideways      


'Straighten hips sideways'

Now, without any uppercase and just with the important words...

In [10]:
prefix = newgen([90,"knees","squat"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

knees bend down      


['Bend knees down', 'Bend knees backwards', 'Bend knees upwards']

# Online Sources for this project

In [11]:
# https://gist.github.com/jlln/338b4b0b55bd6984f883 modified to keep punctuation
# kaggle google store competition for json read
# https://www.kaggle.com/hamishdickson/weighted-word-autocomplete-using-star-wars-dataset

In [1]:
excercise={
    
    "squat":{
        "hips":
    {
    "status":1,    
    "key":["bend","straighten"],
    "dir":["down","vertically"],
    "angles":[70,90]        
    },
    "knees":
    {
    "status":0,    
    "key":["bend","straighten"],
    "dir":["down","vertically"],
    "angles":[20,70]        
    }
    
    },
    "sidekick":{
        "knees":
    {
    "status":1,    
    "key":["bend","straighten"],
    "dir":["down","sideways"],
    "angles":[170,180]        
    }
    
    }
}

In [10]:
excercise["sidekick"]["knees"]["dir"]

['down', 'sideways']

In [8]:
def newgen(data):
    angle=data[0]
    joint=data[1]
    exc=data[2]
    status=excercise[exc][joint]["status"]
    if status==0:
        if angle>excercise[exc][joint]["angles"][0]:
            keywords=joint+" "+excercise[exc][joint]["key"][status]+" "+excercise[exc][joint]["dir"][status]
        else:
            keywords="good"
    else:
        if angle<excercise[exc][joint]["angles"][1]:
            keywords=joint+" "+excercise[exc][joint]["key"][status]+" "+excercise[exc][joint]["dir"][status]
        else:
            keywords="good"

            
    return keywords
    

In [12]:
newgen([92,"knees","sidekick"])

'knees straighten sideways'